## **Hani Soni**

## **202318044**

Task C: Construct a dataset by scraping a website online using BeautifulSoup.

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
from rich import print
import pandas as pd


def get_page_source(url):
    pages = []
    options = Options()
    # options.add_argument("--headless=true")
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get(url)
    time.sleep(10)
    search_box = driver.find_element(
        "xpath", "//input[@id='twotabsearchtextbox']")
    search_box.send_keys("Samsung Mobile")
    search_box.send_keys(Keys.ENTER)
    time.sleep(2)
    page_source = driver.page_source
    pages.append(page_source)
    soup = BeautifulSoup(page_source, "html.parser")
    i = 1
    products = []
    products.extend(scrape_data(page_source))
    while i < 20:
        next_button = soup.find("a", attrs={
                                "class": "s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"})
        if next_button:
            i += 1
            time.sleep(2)  # Add a delay to allow the next page to fully load
            try:
                driver.find_element(
                    "xpath", "//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']").click()
                time.sleep(5)  # Add a delay after clicking the next button
                page_source = driver.page_source
                # Update soup with new page content
                soup = BeautifulSoup(page_source, "html.parser")
                p = scrape_data(page_source)
                if p:
                    products.extend(p)
            except Exception as e:
                print(f"Error: {e}")
                break
    driver.quit()
    print(len(products))
    return products


def scrape_data(page):
    products = []

    soup = BeautifulSoup(page, "html.parser")
    cards = soup.find_all(
        "div", attrs={"data-component-type": "s-search-result"})
    BASE_URL = "https://www.amazon.in"
    for card in cards:
        product = {}
        image_tag = card.find(
            "img", attrs={"data-image-latency": "s-product-image"}) or ""
        img_link = image_tag and image_tag["src"] or ""
        title = card.find("a", attrs={
            "class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"}) or ""
        url = title and BASE_URL + title["href"]
        title = title and title.get_text()
        price = card.find(
            'span', class_='a-price').find('span', class_='a-offscreen') or ""
        price = price and price.get_text()
        delivery_info = card.find(
            "div", attrs={"data-cy": "delivery-recipe"}) or ""
        delivery_info = delivery_info and delivery_info.get_text().strip()
        print(card.get_text())

        product["title"] = title
        product["img_url"] = img_link
        product["url"] = url
        product["price"] = price
        product["delivery_info"] = delivery_info
        products.append(product)

    return products


if __name__ == "__main__":
    url = "https://www.amazon.in/"
    products = get_page_source(url)
    df = pd.DataFrame(products)
    df.to_excel("amazon_products.xlsx")
